# Analísis Exploratorio de los Datos

In [2]:
import pandas as pd


In [3]:
dataset = pd.read_csv('../data/data_transactions.csv')

In [5]:
dataset.head()

,user_id,transaction_type,amount,balance_before,balance_after,timestamp,is_anomaly
0,user_51,recarga,19821.54,474314.99,494136.53,2023-01-01 00:00:00,0
1,user_92,recarga,5743.16,217815.04,223558.20,2023-01-01 00:01:00,0
2,user_14,retiro,17011.92,11710.15,-5301.77,2023-01-01 00:02:00,0
3,user_71,recarga,26655.67,469606.25,496261.92,2023-01-01 00:03:00,0
4,user_60,recarga,33147.40,16918.13,50065.53,2023-01-01 00:04:00,0


In [6]:
dataset.tail()

,user_id,transaction_type,amount,balance_before,balance_after,timestamp,is_anomaly
9995,user_15,recarga,38723.21,400275.82,438999.03,2023-01-07 22:35:00,0
9996,user_16,recarga,13939.31,308558.02,322497.33,2023-01-07 22:36:00,0
9997,user_79,retiro,19073.72,440061.75,420988.03,2023-01-07 22:37:00,0
9998,user_18,retiro,8625.07,359459.44,350834.37,2023-01-07 22:38:00,0
9999,user_7,recarga,6982.52,387260.83,394243.35,2023-01-07 22:39:00,0


In [7]:
dataset.describe()

,amount,balance_before,balance_after,is_anomaly
count,1.000000e+04,10000.000000,1.000000e+04,10000.000000
mean,4.845413e+04,254697.230031,2.672200e+05,0.010400
std,3.154764e+05,141770.476819,3.483824e+05,0.101454
min,1.000000e+03,10008.200000,-4.621451e+06,0.000000
25%,1.168429e+04,130533.632500,1.365646e+05,0.000000
50%,1.818234e+04,254952.050000,2.628995e+05,0.000000
75%,2.474222e+04,377613.022500,3.852559e+05,0.000000
max,4.995116e+06,499986.350000,5.417279e+06,1.000000


Aquí se debe tener en cuenta que solo un 1% de los datos son anomalías, por lo tanto se debe tener cuidado a la hora de splitear los datos para hacer random forest u otros de forma aleatoria, ya que puede que varios árboles no tengan ningún ejemplo de anomalía o estén heavily biased por unos pocos datapoints en el dataset. 

In [10]:
dataset.describe(include="object")

,user_id,transaction_type,timestamp
count,10000,10000,10000
unique,100,2,10000
top,user_16,recarga,2023-01-01 00:00:00
freq,126,6105,1


Solo hay dos tipos de transacciones. Recargas y retiros. Hay un total de 100 usuarios, de los cuales el usuario 16 tiene la mayor cantidad de transacciones realizadas.

In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           10000 non-null  object 
 1   transaction_type  10000 non-null  object 
 2   amount            10000 non-null  float64
 3   balance_before    10000 non-null  float64
 4   balance_after     10000 non-null  float64
 5   timestamp         10000 non-null  object 
 6   is_anomaly        10000 non-null  int64  
dtypes: float64(3), int64(1), object(3)
memory usage: 547.0+ KB


In [ ]:
# Double check de que no existan valores vacíos.
# Efectivamente es un dataset ideal
missing_data = dataset.isnull().sum()
missing_percentage = (missing_data / len(dataset)) * 100

missing_percentage

user_id             0.0
transaction_type    0.0
amount              0.0
balance_before      0.0
balance_after       0.0
timestamp           0.0
is_anomaly          0.0
dtype: float64

No hay datos vacíos en el dataset. Por lo que no debemos aplicar ninguna técnica para llenarlos.

## Feature Engineering

Dado que no hay ningún dato vacío, y no hay un indicador claro de que variables pueden ser features fuertes para un modelo predictivo, vamos a aislar las anomalías para ver si tienen una diferencia significativa con los datos reales y podemos tener una idea general de que variables puden ser strong predictors para nuestro modelo. 



In [15]:
anomaly = dataset[dataset['is_anomaly'] == 1]
legitimate = dataset[dataset['is_anomaly'] == 0]

In [16]:
anomaly.head()

,user_id,transaction_type,amount,balance_before,balance_after,timestamp,is_anomaly
7,user_86,recarga,2438737.50,240491.47,2679228.97,2023-01-01 00:07:00,1
84,user_61,recarga,4970018.15,447261.25,5417279.40,2023-01-01 01:24:00,1
191,user_96,retiro,4916307.24,377886.08,-4538421.16,2023-01-01 03:11:00,1
199,user_26,recarga,4194378.53,409143.66,4603522.19,2023-01-01 03:19:00,1
253,user_72,recarga,2370883.11,38094.03,2408977.14,2023-01-01 04:13:00,1


In [17]:
legitimate.head()

,user_id,transaction_type,amount,balance_before,balance_after,timestamp,is_anomaly
0,user_51,recarga,19821.54,474314.99,494136.53,2023-01-01 00:00:00,0
1,user_92,recarga,5743.16,217815.04,223558.20,2023-01-01 00:01:00,0
2,user_14,retiro,17011.92,11710.15,-5301.77,2023-01-01 00:02:00,0
3,user_71,recarga,26655.67,469606.25,496261.92,2023-01-01 00:03:00,0
4,user_60,recarga,33147.40,16918.13,50065.53,2023-01-01 00:04:00,0


In [18]:
anomaly.describe()

,amount,balance_before,balance_after,is_anomaly
count,1.040000e+02,104.000000,1.040000e+02,104.0
mean,2.915214e+06,264526.082788,8.548086e+05,1.0
std,1.126014e+06,143409.941577,3.077410e+06,0.0
min,1.014924e+06,14740.440000,-4.621451e+06,1.0
25%,2.095876e+06,161358.327500,-2.203949e+06,1.0
50%,2.850572e+06,275176.060000,1.919371e+06,1.0
75%,3.794361e+06,382729.742500,3.411152e+06,1.0
max,4.995116e+06,495739.980000,5.417279e+06,1.0


In [20]:
anomaly.describe(include="object")

,user_id,transaction_type,timestamp
count,104,104,104
unique,58,2,104
top,user_24,recarga,2023-01-01 00:07:00
freq,5,64,1


In [19]:
legitimate.describe()

,amount,balance_before,balance_after,is_anomaly
count,9896.000000,9896.000000,9896.000000,9896.0
mean,18326.498404,254593.935701,261044.907727,0.0
std,9390.763639,141756.856896,142822.542040,0.0
min,1000.000000,10008.200000,-25366.290000,0.0
25%,11605.377500,130247.722500,137064.540000,0.0
50%,18042.915000,254376.935000,262565.670000,0.0
75%,24513.722500,377513.167500,383571.500000,0.0
max,58067.170000,499986.350000,539116.140000,0.0


In [22]:
legitimate.describe(include="object")

,user_id,transaction_type,timestamp
count,9896,9896,9896
unique,100,2,9896
top,user_16,recarga,2023-01-01 00:00:00
freq,125,6041,1


De este análsis es claro que las transacciones fraudulentas mueven en promedio cantidades mucho mas altas de dinero. Por lo tanto la cantidad de dinero asociada a la transacción va a ser el indicador más fuerte en cualquier modelo que entrenemos con los datos, especialemente si se trata de árboles de decisión. 

Posibles problemas de esta tendencia estadística en la vida real: 
 - Un posible problema significativo de esta tendencia en la vida real es que va a ser muy probable que las transcacciones hechas por big players o ballenas en la plataforma sean flageadas cómo anomalas. Idealmente deberíamos tener una feature que contrareste este efecto.
 
 En el caso de los retiros: 
- Si una transacción es fraudulenta, es muy probable que la transacción se haga por una gran cantidad de dinero, por lo cual el saldo final va a tender a ser muy negativo. Esto también es un strong feature pero tiene una fuerte correlación con la cantidad de la transacción.

También hay usuarios que solo tienen transacciones legítimas, por lo que dado el tamaño relativamente pequeño del dataset puede generar bias en el modelo inicial. 